In [ ]:
import io, os, re, sys
import numpy as np
import pandas as pd
import pytesseract
import fitz
import PyPDF2

# Table extraction: Docling(doc:https://docling-project.github.io/docling/), tabula


from dotenv import load_dotenv
from PIL import Image

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            google_api_key=os.getenv("GOOGLE_API_KEY"), 
            temperature=0.1
        )
embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=os.getenv("GOOGLE_API_KEY")
        ) 

In [ ]:
print(sys.version)
print(sys.executable)

In [ ]:
os.listdir('data/')

# Extract data from PDF

In [41]:
q1_report_path = "data/raw/2025_1Q_LGES_Audit_Report_CONFS_en.pdf"

In [ ]:
docs = fitz.open(q1_report_path)
print(f"total pages: {len(docs)}")

### Docling test

In [1]:
from docling.document_converter import DocumentConverter

In [2]:
from pathlib import Path

In [6]:
doc_converter = DocumentConverter()
conv_res = doc_converter.convert(q1_report_path)

2025-09-09 19:41:09,657 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-09 19:41:09,782 - INFO - Going to convert document batch...
2025-09-09 19:41:09,785 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 409740a308d273b3f090712d250cd783
2025-09-09 19:41:09,865 - INFO - Loading plugin 'docling_defaults'
2025-09-09 19:41:09,873 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-09 19:41:51,446 - INFO - Accelerator device: 'cpu'
2025-09-09 19:41:51,453 - WARNING - Downloading detection model, please wait. This may take several minutes depending upon your network connection.
2025-09-09 19:42:02,585 - INFO - Download complete
2025-09-09 19:42:02,586 - WARNING - Downloading recognition model, please wait. This may take several minutes depending upon your network connection.
2025-09-09 19:42:05,160 - INFO - Download complete.
2025-09-09 19:42:07,534 - INFO - Accelerator device: 'cpu'
2025-09-09 19:43:03,8

In [7]:
doc_filename = conv_res.input.file.stem

In [8]:
doc_filename

'2025_1Q_LGES_Audit_Report_CONFS_en'

In [15]:
dfs = {}
for table_i, table in enumerate(conv_res.document.tables):
    table_df: pd.DataFrame = table.export_to_dataframe()
    # print(table_df.to_markdown())
    dfs[f"tabel_{table_i}"] = table_df

----

In [ ]:
print(docs, type(docs))
print(docs[0], type(docs[0]))

In [ ]:
def is_scanned_page(page):
    """
    If pages are covered with only images, run OCR.

    ??? What if half is filled with images and half with text?
    """
    text = page.get_text().strip()
    images = page.get_images()
    return len(images) > 0 and len(text) < 25 

def extract_text_ocr(page):
    pix = page.get_pixmap()
    img_data = pix.tobytes("png")
    img = Image.open(io.BytesIO(img_data))

    # text extraction using OCR
    text = pytesseract.image_to_string(img)
    return text

def clean_text(text):
    """Clean extracted text"""
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters but keep punctuation
    text = re.sub(r'[^\w\s\-.,;:!?()]', '', text)
    return text.strip()

In [ ]:
full_text = ""
for pg_i, page in enumerate(docs):
    if is_scanned_page(page):
        ocr_text = extract_text_ocr(page)
        if ocr_text.strip():
            page_text = ocr_text
        else:
            print(f"--------OCR failed------- at page:{pg_i}")
    else:
        page_text = page.get_text()
        
    page_text = clean_text(page_text)
    full_text += f"\n--- Page {pg_i + 1} ---\n{page_text}\n"
    
with open('data/processed/2025_1Q_LGES_Audit_Report_CONFS_en_fulltext.txt', 'w') as f:
   f.write(full_text) 

In [ ]:
# ??? Extract Table of content: So it's information can be leveraged somewher(ex: chunk by chapter, etc..)
# table_of_contents = doc[1].get_text()

# ??? Better way to extract table? and standardize schema across various reports from diff countries.
# table = doc[5].get_text() + "\n" + doc[6].get_text()

## when PDF contains too many tables

Unless we use Multi-modal embedding model, and extracting tables work perfectly (Images seem fine but if table in pdf is hard to comprehend, even for human) it is better to add another chain, that is, ask LLM to generate a summary then summary becomes your texts to embed (or could feed it directly).

In [34]:
text_gen_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.1
)

In [ ]:
text_gen_model.file

In [42]:
from google import genai

client = genai.Client()

myfile = client.files.upload(file=q1_report_path)
myfile

2025-09-09 21:23:47,108 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
2025-09-09 21:23:48,437 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=ABgVH88Svkldc6NTYAr22eN0aCOh64bNcKd7-wMdiFv5wSQvc1hZXoGz-04KY0PvULoiLqrVAwoiv9E622AOWkxxwyrne_rnmOZcHU1_RjRQBw&upload_protocol=resumable "HTTP/1.1 200 OK"


In [55]:
from google import genai
from google.genai import types
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

client = genai.Client()
system_message = """
You are a financial analyst who have been following 
LG energy solution for few years.
You are extremely knowlegeable about battery industry.
From given financial report of LG energy solution, 
Create a summary report that will help investors make decisions 
on when to buy on the stock market.
"""

# Upload file
file = client.files.upload(file=q1_report_path)
while file.state.name == 'PROCESSING':
    time.sleep(2)
    file = client.files.get(name=file.name)

# Create cache
# model = 'models/gemini-1.5-flash-latest'
# cache = client.caches.create(
#     model=model,
#     config=types.CreateCachedContentConfig(
#         display_name='Cached Content',
#         system_instruction=(system_message),
#         contents=[file],
#         ttl="300s",
#     )
# )


message = HumanMessage(content="")
response = text_gen_model.invoke([system_message])

2025-09-09 21:50:25,427 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
2025-09-09 21:50:26,758 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=ABgVH89wrbad03g1Q7wet9M6rewu5vrvW4XuZISvA0L6iyqijtujcTuifKmDOJz1IrRUOQogdXUnCK3u7PlvHKQVO5_EHLbnjJnFE7IBAxVDGA&upload_protocol=resumable "HTTP/1.1 200 OK"


### **Executive Summary**

LG Energy Solution (LGES) stands as a global titan in the rapidly expanding battery industry, a critical enabler of the electric vehicle (EV) and energy storage system (ESS) revolution. Our long-term conviction in LGES remains strong, driven by its technological leadership, diversified customer base, aggressive capacity expansion, and strategic positioning to benefit from global decarbonization efforts.

However, the stock's valuation has historically commanded a premium, reflecting its growth prospects. **For investors looking to initiate or increase positions, our analysis suggests a patient and tactical approach, focusing on specific market conditions and operational catalysts rather than chasing momentum.** We believe optimal entry points will emerge during periods of market consolidation, temporary operational headwinds, or when the market underappreciates LGES's long-term strategic advantages and improving profitability profile.

---

### **Company Overview**

LG Energy Solution, spun off from LG Chem in 2020, is a leading global manufacturer of lithium-ion batteries for electric vehicles (EVs), energy storage systems (ESS), and small-format applications. With a diverse product portfolio including pouch, cylindrical, and prismatic cells, LGES boasts an impressive roster of global OEM clients such as General Motors, Hyundai, Stellantis, Volkswagen, Tesla, Ford, Lucid, BMW, and Mercedes-Benz. The company is aggressively expanding its global manufacturing footprint, particularly in North America, to capitalize on the Inflation Reduction Act (IRA) benefits and meet surging demand.

---

### **Key Financial Highlights (Based on Recent Trends & Industry Expectations)**

*   **Revenue Growth:** Consistently strong, driven by robust EV demand and significant capacity additions. We've observed double-digit to triple-digit percentage growth year-over-year, though the pace may normalize slightly as the base grows larger.
*   **Gross Margins:** Have shown volatility. Initial ramp-up costs for new facilities, fluctuating raw material prices (lithium, nickel, cobalt), and initial pricing pressure from OEMs have impacted profitability. However, we anticipate gradual improvement as new plants achieve economies of scale, raw material costs stabilize, and the benefits of the IRA (Advanced Manufacturing Production Credit, 45X) begin to fully materialize.
*   **Operating Profit:** Reflects the gross margin trends, further impacted by substantial R&D investments and SG&A expenses associated with global expansion.
*   **Capital Expenditure (CapEx):** Extremely high, indicative of the company's aggressive capacity expansion plans. This is a necessary investment for future growth but will continue to weigh on free cash flow in the near to medium term.
*   **Cash Flow:** Operating cash flow is generally positive, but free cash flow often remains negative due to the massive CapEx requirements. The company has managed its balance sheet effectively post-IPO, but debt levels are likely to increase to fund expansion.
*   **Guidance:** Management typically provides ambitious revenue targets and outlines significant CapEx plans, while cautiously guiding on profitability, acknowledging the dynamic raw material and competitive landscape.

---

### **Industry & Competitive Landscape**

The battery industry is characterized by explosive growth, intense competition, and rapid technological evolution.

*   **Growth Drivers:** Global EV adoption, government mandates for decarbonization, and the increasing need for grid-scale ESS are powerful tailwinds.
*   **Competition:** LGES faces formidable rivals, primarily CATL and BYD (especially in China), Samsung SDI, SK On, and Panasonic. Each player is aggressively expanding and innovating.
*   **Raw Materials:** Volatility in key raw material prices (lithium, nickel, cobalt, graphite) remains a significant factor influencing profitability and supply chain stability. LGES is actively pursuing vertical integration and long-term supply agreements.
*   **Technological Shifts:** While NMC (Nickel-Manganese-Cobalt) remains dominant for performance EVs, LFP (Lithium Iron Phosphate) is gaining traction for mass-market vehicles due to cost advantages. Solid-state batteries represent a long-term disruptive potential, with LGES actively investing in R&D.
*   **Regionalization:** Policies like the U.S. IRA and similar initiatives in Europe are driving localized manufacturing and supply chains, benefiting players with strong regional footprints like LGES.

---

### **Strengths & Opportunities**

1.  **Market Leadership & Diversified Portfolio:** A top-tier global player with a broad product range and a robust, diversified customer base across multiple geographies.
2.  **North American Expansion & IRA Benefits:** LGES is exceptionally well-positioned to capitalize on the IRA's production tax credits (45X) through its numerous joint ventures (JVs) and wholly-owned facilities in the U.S. This provides a significant competitive advantage and a substantial boost to future profitability.
3.  **Technological Prowess:** Strong R&D capabilities, evidenced by continuous improvements in energy density, charging speed, and safety. Active development in next-generation battery technologies.
4.  **ESS Market Growth:** Beyond EVs, the ESS market is poised for exponential growth, driven by renewable energy integration and grid stabilization. LGES is a leading player here, offering another significant growth vector.
5.  **Vertical Integration & Supply Chain Resilience:** Proactive efforts to secure raw material supply and explore recycling initiatives enhance long-term stability and cost control.

---

### **Risks & Challenges**

1.  **Intense Competition & Pricing Pressure:** The aggressive expansion of competitors, particularly Chinese players, could lead to pricing pressure and potential oversupply in certain segments or regions in the long run.
2.  **Raw Material Volatility:** Despite mitigation efforts, significant fluctuations in raw material prices can compress margins.
3.  **High CapEx & Free Cash Flow Strain:** The massive investment required for capacity expansion will continue to consume significant capital, potentially limiting free cash flow generation in the near term.
4.  **Technology Transition Risk:** While LGES is investing in LFP and solid-state, a faster-than-expected shift away from its core NMC technology could pose challenges.
5.  **Geopolitical Risks:** Trade tensions, supply chain disruptions, and regulatory changes (e.g., U.S.-China relations) could impact operations and profitability.
6.  **Operational Execution:** Successfully ramping up numerous new, complex manufacturing facilities globally without significant delays or cost overruns is a continuous challenge.
7.  **Product Recalls:** While past issues (e.g., GM Bolt, Hyundai Kona) have largely been resolved, any future large-scale recalls could damage reputation and incur significant costs.

---

### **Valuation & Investment Thesis**

LGES has historically traded at a premium valuation (e.g., high Price-to-Sales, Price-to-Earnings multiples) reflecting its market leadership and robust growth trajectory. Our investment thesis is predicated on:

*   **Long-Term Structural Growth:** The electrification trend is irreversible, providing a multi-decade growth runway for battery manufacturers.
*   **IRA-Driven Profitability Boost:** The 45X tax credits are a game-changer, significantly enhancing LGES's profitability profile, particularly from its North American operations, which will become increasingly dominant.
*   **Operational Leverage & Scale:** As new plants mature and achieve higher utilization rates, we expect to see improved operating leverage and margin expansion.
*   **Diversification:** Its balanced exposure to EVs, ESS, and various OEM clients provides resilience.

However, the current valuation often prices in much of this future growth. Therefore, **the key for investors is not *if* LGES will grow, but *at what price* to participate in that growth.**

---

### **Recommendation & Entry Points: When to Buy**

We maintain a **"Buy on Dips with Long-Term Conviction"** rating for LGES. Our recommendation for investors looking to enter or increase their position is to be patient and strategic, targeting the following scenarios:

1.  **Market-Wide Corrections:** General equity market downturns or sector-specific pullbacks (e.g., EV sector cooling off) often present opportunities to acquire high-quality growth stocks like LGES at more attractive valuations. Look for periods where the broader market sentiment is negative, but LGES's fundamental outlook remains intact.
2.  **LGES-Specific Operational Headwinds (Non-Structural):**
    *   **Temporary Margin Pressure:** Short-term dips due to higher-than-expected raw material costs, initial ramp-up inefficiencies at new plants, or temporary pricing adjustments. These are often transient and do not undermine the long-term growth story.
    *   **Guidance Misses (Short-Term):** If LGES misses quarterly earnings or revenue guidance due to temporary factors (e.g., supply chain bottlenecks, customer inventory adjustments), the stock may pull back, offering an entry point.
3.  **Valuation Compression:** Monitor key valuation multiples (e.g., EV/Sales, P/S, P/E relative to growth). If the stock's multiples compress significantly below its historical average or relative to its closest peers (adjusting for IRA benefits), it could signal an attractive entry. We would look for a Price-to-Sales ratio that reflects a more reasonable discount to its projected 3-5 year revenue growth.
4.  **Confirmation of IRA Benefits & Margin Expansion:** As LGES begins to report clearer and more substantial benefits from the IRA (e.g., specific 45X credit figures, noticeable improvement in North American segment margins), the market may re-rate the stock. Buying *before* these benefits are fully priced in, perhaps during a period of skepticism, could be advantageous.
5.  **Technical Support Levels:** For technically-minded investors, identifying strong historical support levels on the stock chart can provide tactical entry points during pullbacks.

**What to Monitor for Entry:**

*   **Raw Material Price Stability:** A sustained period of stable or declining raw material costs would be a significant positive for margins.
*   **New Order Announcements:** Major new OEM contracts or expansions of existing ones, especially for next-generation platforms.
*   **CapEx Efficiency:** Signs that new plants are ramping up smoothly and on budget, leading to improved utilization rates.
*   **Competitive Landscape:** Any signs of competitors facing significant setbacks or LGES gaining market share.
*   **Management Commentary:** Listen closely to earnings calls for insights into margin outlook, IRA credit realization, and demand trends.

**In essence, LGES is a long-term hold for exposure to the electrification megatrend. However, given its premium valuation, investors should exercise patience and look for opportunities to buy when the market offers a discount, rather than chasing upward momentum. The periods of temporary weakness or market skepticism are often the best times to build a position in a fundamentally strong, growth-oriented company like LGES.**

---

**Disclaimer:** This report is for informational purposes only and does not constitute financial advice. Investors should conduct their own due diligence and consult with a qualified financial advisor before making any investment decisions. The views expressed herein are based on current market conditions and information available at the time of writing and are subject to change without notice. Investing in the stock market involves risks, including the potential loss of principal.

In [51]:
import base64
from langchain_core.messages import HumanMessage

pdf_bytes = open(q1_report_path, 'rb').read()
pdf_base64 = base64.b64encode(pdf_bytes).decode('utf-8')

message = HumanMessage(
    content=[
        {"type": "text", "text": "You are a financial analyst who have been following LG energy solution for few years. You are extremely knowlegeable about battery industry. From given financial report of LG energy solution, Create a summary report that will help investors make decisions on when to buy on the stock market."},
        {
            "type": "file",
            "source_type": "base64",
            "mime_type":"application/pdf",
            "data": pdf_base64
        }
    ]
)

response = text_gen_model.invoke([message])

## LG Energy Solution (LGES) Investor Summary Report: Q1 2025 Performance & Investment Outlook

**Date:** May 15, 2025
**Analyst:** [Your Name/Firm Name]
**Subject:** LG Energy Solution (LGES) - Q1 2025 Performance Analysis and Investment Recommendation

---

**Executive Summary:**

LG Energy Solution (LGES) delivered a mixed performance in Q1 2025. While the company demonstrated robust operational growth, significantly boosted by the U.S. Inflation Reduction Act (IRA) tax credits, the net profit attributable to parent company shareholders recorded a loss, which worsened year-over-year. This divergence is primarily due to the substantial share of profits allocated to non-controlling interests in its joint ventures (JVs) and increased income tax expense.

LGES remains in an aggressive expansion phase, particularly in North America, evidenced by continued heavy capital expenditures (CAPEX) and rising debt levels. The long-term outlook for the battery industry, driven by EV and ESS demand, remains strong, and LGES is strategically positioned to capitalize on this growth through its extensive global footprint and JV partnerships.

**For investors, the current period represents a critical juncture of high investment and future potential. Buying opportunities are likely to emerge when there is clearer evidence of the JVs' profitability translating more directly to LGES's bottom line, or during market pullbacks that do not reflect the underlying operational strength and IRA tailwinds.**

---

**Q1 2025 Financial Highlights (vs. Q1 2024):**

*   **Revenue:** Increased by a modest **2.22%** to ₩6,264,995 million (from ₩6,128,714 million). This indicates continued, albeit slower, top-line expansion in a competitive market.
*   **Gross Profit:** Grew by **9.55%** to ₩1,012,256 million (from ₩923,999 million), suggesting an improvement in gross margins.
*   **Other Operating Income:** A significant surge of **142.2%** to ₩457,716 million (from ₩188,954 million). **Crucially, this includes substantial benefits from the U.S. IRA Advanced Manufacturing Production Tax Credit, a key driver of operational profitability.**
*   **Operating Profit:** Skyrocketed by **138.1%** to ₩374,673 million (from ₩157,337 million). This impressive growth is largely attributable to the IRA tax credits, masking some underlying cost increases in SG&A.
*   **Selling and Administrative Expenses (SG&A):** Increased by **14.6%** to ₩1,095,299 million (from ₩955,616 million), outpacing revenue growth and indicating ongoing investment in sales, marketing, and administrative functions.
*   **Profit for the Period Attributable to Owners of the Parent Company:** A **loss of ₩145,740 million**, worsening from a loss of ₩464 million in Q1 2024. This is a critical point for investors.
    *   **Non-controlling Interests:** A substantial portion of the overall profit (₩372,313 million in Q1 2025 vs. ₩212,516 million in Q1 2024) was allocated to non-controlling interests, primarily from profitable joint ventures like Ultium Cells. This means the JVs are generating profits, but LGES shareholders only see their proportionate share *after* non-controlling interests are accounted for.
    *   **Income Tax Expense:** Increased significantly to ₩137,933 million (from ₩10,597 million), further impacting net income.
    *   **Finance Costs:** Increased, and exchange differences resulted in a net loss in Q1 2025 compared to a gain in Q1 2024, contributing to the negative bottom line for the parent company.
*   **Basic Loss Per Share:** Worsened to (₩623) from (₩2) in Q1 2024, reflecting the net loss for parent company shareholders.

---

**Financial Position & Cash Flow Analysis:**

*   **Assets:** Total assets grew to ₩62,298,978 million (from ₩60,306,791 million at YE 2024), driven by significant investments in Property, Plant & Equipment (PPE) and Intangible Assets. Construction in progress alone stands at ₩22,389,745 million, highlighting the scale of ongoing capacity build-out.
*   **Liabilities & Equity:** Total liabilities increased to ₩31,028,433 million (from ₩29,340,248 million at YE 2024).
    *   **Borrowings:** Total borrowings rose substantially to approximately ₩17.6 trillion (from ₩15.4 trillion at YE 2024), indicating a heavy reliance on debt to fund CAPEX.
    *   **Gearing Ratio & Debt-to-Equity:** Both ratios increased (Gearing: 31.0% from 27.1%; Debt-to-Equity: 99.2% from 94.7%), reflecting the higher debt load. While manageable for a growth company, this trend warrants close monitoring.
    *   **Non-controlling Interests in Equity:** Increased to ₩10,107,900 million (from ₩9,850,306 million at YE 2024), further underscoring the capital contributions from JV partners.
*   **Cash Flow from Operations:** Improved significantly to ₩1,162,033 million (from ₩531,069 million), demonstrating stronger operational cash generation.
*   **Cash Flow from Investing Activities:** Remained a substantial outflow of (₩3,440,825) million, consistent with the aggressive CAPEX strategy.
*   **Cash Flow from Financing Activities:** Provided a net inflow of ₩1,950,253 million, primarily from new borrowings and capital contributions from non-controlling interests, to offset the investing outflows.
*   **Net Cash Position:** Cash and cash equivalents decreased by ₩328,539 million in Q1 2025, indicating that even with strong operating cash flow and financing, the massive investment outlays are consuming cash.

---

**Industry & Strategic Context (Analyst's Perspective):**

LGES is a global leader in the battery industry, with a diversified portfolio spanning automotive, mobile, and ESS applications. The company's strategy is heavily focused on expanding its manufacturing footprint, particularly in North America, to meet the surging demand for electric vehicles (EVs) and energy storage systems (ESS).

1.  **North American Expansion & IRA:** The significant increase in "Other Operating Income" due to IRA tax credits is a powerful tailwind for LGES's U.S. operations. This non-operating income directly boosts profitability and provides a competitive advantage. The numerous JVs (Ultium Cells with GM, JVs with Honda, Stellantis, HMG Global) are central to this strategy, allowing LGES to share the immense CAPEX burden and de-risk some of the investment.
2.  **JV Structure Impact:** The negative net profit attributable to parent company shareholders, despite strong operating profit, is a structural characteristic of LGES's heavy reliance on JVs. A large portion of the JVs' profits is initially allocated to non-controlling interests. While this dilutes reported net income for LGES shareholders, it indicates that the underlying JV businesses are profitable and growing, which is a positive for the overall enterprise value. Investors need to look beyond the headline net income and understand the consolidated structure.
3.  **CAPEX Cycle:** LGES is in a heavy CAPEX cycle, building out gigafactories globally. This is necessary to secure future market share but puts pressure on cash flow and increases debt. The long-term payoff, however, is substantial if demand materializes as expected.
4.  **Market Dynamics:** The battery market is characterized by intense competition, technological advancements, and raw material price volatility. LGES's hedging strategies (e.g., raw material swaps) and diversified customer base help mitigate some of these risks. Foreign exchange fluctuations remain a notable risk, as evidenced by the Q1 2025 finance costs.
5.  **Legal & Regulatory Risks:** Ongoing lawsuits (mobile batteries, GM Bolt EV) and the need to comply with various environmental and trade regulations (e.g., greenhouse gas emission rights) present potential liabilities and operational complexities.

---

**Investment Recommendation: When to Buy LGES Stock**

LGES is a high-growth company operating in a secular growth industry. Its current financial statements reflect a company aggressively investing for future dominance, with significant operational tailwinds (IRA) but also structural complexities (JVs) impacting reported net income to parent shareholders.

**I recommend a "Buy on Dips" strategy for LGES, with a focus on the following triggers and considerations:**

1.  **Market Misinterpretation of Net Income:** The negative net income attributable to parent company shareholders, while structurally explainable by JV accounting, could lead to short-term market overreactions. **A significant dip in stock price following such announcements, if the underlying operating profit (especially with IRA benefits) remains strong, could present an attractive entry point.** Investors should understand that the JVs are profitable, and LGES's share of that profit is substantial, even if it's not fully reflected in the "attributable to parent" line due to non-controlling interests.
2.  **Confirmation of JV Ramp-up and Utilization:** As new JV facilities in North America come online and achieve higher utilization rates, the operational leverage should improve. **Look for future reports that show increasing revenue contribution from these new facilities and, more importantly, a more favorable trend in the profit allocation to LGES's parent company, or a stabilization of the non-controlling interest share relative to total profit.** This would signal that the massive investments are maturing and directly benefiting LGES shareholders.
3.  **Signs of CAPEX Peaking and Improved Free Cash Flow:** While aggressive CAPEX is necessary now, a sustained period of high debt and negative net cash flow can be a concern. **An indication that the most intense phase of capacity expansion is nearing completion, leading to a reduction in CAPEX intensity and a subsequent improvement in free cash flow, would be a strong buy signal.** This would suggest the company is transitioning from a heavy investment phase to a more cash-generative one.
4.  **Favorable Macroeconomic Environment:**
    *   **Interest Rate Stabilization/Reduction:** Lower interest rates would ease the burden of LGES's significant floating-rate debt, improving finance costs.
    *   **Stable Raw Material Prices:** Continued effective hedging and stable raw material markets would support gross margins.
    *   **Sustained EV/ESS Demand:** Strong global adoption rates for EVs and ESS will ensure demand for LGES's expanding capacity.
5.  **Resolution of Major Contingencies:** Positive developments in the ongoing lawsuits could remove an overhang on the stock.

**In summary, LGES is a long-term play on the electrification trend. While the Q1 2025 net loss for parent shareholders might deter some, the underlying operational strength, driven by strategic IRA benefits and aggressive JV expansion, positions the company for future growth. Patient investors should look for opportunities to accumulate shares during market corrections that fail to fully appreciate LGES's strategic positioning and the eventual payoff from its current investment cycle.**

---
**Disclaimer:** This report is for informational purposes only and does not constitute financial advice. Investors should conduct their own due diligence and consult with a qualified financial advisor before making any investment decisions.

In [60]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


llm_with_tools = text_gen_model.bind_tools([GetWeather, GetPopulation])
ai_msg = llm_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

[]

In [62]:
ai_msg.content

"I can't tell you which city is hotter today. I can tell you which is bigger between Los Angeles and New York. What information are you interested in, population or area?"

In [ ]:
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
resp = llm.invoke(
    "When is the next total solar eclipse in US?",
    tools=[GenAITool(google_search={})],
)

## Sending video or youtube to Gemini

In [ ]:
video_bytes = open("/path/to/your/video.mp4", 'rb').read()
video_base64 = base64.b64encode(video_bytes).decode('utf-8')

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe what's in this video in a sentence"},
        {
            "type": "file",
            "source_type": "base64",
            "mime_type": "video/mp4",
            "data": video_base64
        }
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content


message = HumanMessage(
    content=[
        {"type": "text", "text": "summarize the video in 3 sentences."},
        {
            "type": "media",
            "file_uri": "https://www.youtube.com/watch?v=9hE5-98ZeCg",
            "mime_type": "video/mp4",
        }
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content